In [1]:
import pandas as pd
import numpy as np

pm = pd.read_csv('../govdata_15min_LodhiRoad_IMD.csv') # pollution data (only for Lodhi)
pm['timestamp_round'] = pd.to_datetime(pm['timestamp_round'])
pm.head()

,monitor_id,timestamp_round,pm25,pm10
0,LodhiRoad_IMD,2018-03-01 05:30:00+05:30,98.21,187.84
1,LodhiRoad_IMD,2018-03-01 05:45:00+05:30,94.63,193.80
2,LodhiRoad_IMD,2018-03-01 06:00:00+05:30,92.76,198.77
3,LodhiRoad_IMD,2018-03-01 06:15:00+05:30,99.30,221.69
4,LodhiRoad_IMD,2018-03-01 06:30:00+05:30,100.16,229.79


In [55]:
insta = pd.read_csv('../insta_delhi_data - Sheet1.csv') # insta data
insta['datetime'] = pd.to_datetime(insta['datetime'])
insta['dt_round'] = insta['datetime'].dt.round('15min')
insta.head()

,filename,datetime,location,file_format,light/dark,dt_round
0,50294736_435828830291236_2030158088675541642_n,2019-02-04 03:48:43+00:00,new delhi,jpg,NaN,2019-02-04 03:45:00+00:00
1,23279786_148197742578856_6378254909640278016_n,2017-11-08 13:24:15+00:00,NaN,jpg,NaN,2017-11-08 13:30:00+00:00
2,23416386_1840710242870758_1418351604486111232_n,2017-11-08 13:24:15+00:00,NaN,jpg,NaN,2017-11-08 13:30:00+00:00
3,23279242_754613681413241_2636689794633039872_n,2017-11-08 12:30:04+00:00,NaN,jpg,NaN,2017-11-08 12:30:00+00:00
4,23421744_157913204814458_5150676316399337472_n,2017-11-11 02:39:00+00:00,NaN,jpg,NaN,2017-11-11 02:45:00+00:00


####TODO: convert timezone in poll data
####TODO: combine pollution and image data
####TODO: location-wise pollution
####TODO: beijing/shanghai data
####TODO: pass through LeNet
####TODO: some basic preprocessing then LeNet

In [56]:
from os import listdir

def unroll_folder(foldername):
    global insta
    files = listdir('../img/'+foldername)
    files = [i[:-4] for i in files]
    x = insta.loc[insta['filename'] == foldername]
    idx = x.index.values[0]
    x = pd.concat([x]*len(files), ignore_index=True)
    for index, row in x.iterrows():
        x.set_value(index, 'filename', row['filename']+'/'+files[index])
    insta = insta.drop([idx])
    insta = insta.append(x, ignore_index=True)

    
unroll_folder('DelhiRoadPhotos76')
unroll_folder('23photos')
insta.tail()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()


,filename,datetime,location,file_format,light/dark,dt_round
911,23photos/5,2018-08-17 14:30:00+00:00,NaN,NaN,NaN,2018-08-17 14:30:00+00:00
912,23photos/7,2018-08-17 14:30:00+00:00,NaN,NaN,NaN,2018-08-17 14:30:00+00:00
913,23photos/6,2018-08-17 14:30:00+00:00,NaN,NaN,NaN,2018-08-17 14:30:00+00:00
914,23photos/2,2018-08-17 14:30:00+00:00,NaN,NaN,NaN,2018-08-17 14:30:00+00:00
915,23photos/1,2018-08-17 14:30:00+00:00,NaN,NaN,NaN,2018-08-17 14:30:00+00:00


In [57]:
import cv2 

def vid2frame(filename): 
    vidObj = cv2.VideoCapture(filename+'.mp4') 
    count = 0
    count2 = 0
    success = 1
    while success:
        success, image = vidObj.read()
        #print(filename+'.mp4')
        if count%30 == 0:
            cv2.imwrite(filename+"frame%d.jpg" % count2, image)
            count2 += 1
        count += 1
    return count2

vids = insta.loc[insta['file_format'] == 'mp4']
for index, row in vids.iterrows():
    nframes = vid2frame('../img/'+row['filename'])
    x = insta.loc[insta['filename'] == row['filename']]
    idx = x.index.values[0]
    x = pd.concat([x]*nframes, ignore_index=True)
    for index2, row2 in x.iterrows():
        x.set_value(index2, 'filename', row2['filename']+'frame'+str(index2))
        x.set_value(index2, 'file_format', 'jpg')
    insta = insta.drop([idx])
    insta = insta.append(x, ignore_index=True)

insta.tail()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,filename,datetime,location,file_format,light/dark,dt_round
2379,41405288_1371374499660316_6484544909150781440_...,2018-09-13 20:19:46+00:00,NaN,jpg,NaN,2018-09-13 20:15:00+00:00
2380,41405288_1371374499660316_6484544909150781440_...,2018-09-13 20:19:46+00:00,NaN,jpg,NaN,2018-09-13 20:15:00+00:00
2381,41405288_1371374499660316_6484544909150781440_...,2018-09-13 20:19:46+00:00,NaN,jpg,NaN,2018-09-13 20:15:00+00:00
2382,41405288_1371374499660316_6484544909150781440_...,2018-09-13 20:19:46+00:00,NaN,jpg,NaN,2018-09-13 20:15:00+00:00
2383,41405288_1371374499660316_6484544909150781440_...,2018-09-13 20:19:46+00:00,NaN,jpg,NaN,2018-09-13 20:15:00+00:00


In [58]:
insta.to_csv('../insta_data.csv',index=False)